# 迭代器

## 简介

在**Ruby**中，迭代器是允许循环遍历集合中的所有元素一个方法，它允许传入一个回调的代码块。迭代器方法只负责遍历需要遍历的元素，而对元素所做的处理则通过回调代码块来实现。

常见的迭代器方法有`each`，`detect`(aka `find`)，`collect`(aka `map`)，`inject`(aka `reduce`)。

像这种，遍历集合的控制权在迭代器方法上，它将需要处理的值推送给需要它们的代码块。我们称这种迭代器为 **内部迭代器**。

In [1]:
a = [2, 4, 6]

a.each do|n|
  puts n
end

2
4
6


[2, 4, 6]

其好处是不需要对下标进行迭代，但是有些情况下，我们既希望获得下标，也希望获得对应的值，那么可以使用 `each_with_index` 函数，这样每次迭代都会返回一组 `(value, index)` 组成的数组：

In [2]:
a = [2, 4, 6]

a.each_with_index do|n, i|
  puts "pos #{i} is #{n}"
end

pos 0 is 2
pos 1 is 4
pos 2 is 6


[2, 4, 6]

在 **Ruby** 中，你依然可以很轻松地使用 `for in` 语句来遍历集合，只要集合实现了 `each` 方法。

In [3]:
a = [2, 4, 6]

for n in a
  puts n
end

2
4
6


[2, 4, 6]

## 自定义迭代器

下面我们来看一个例子，如何实现一个支持迭代器方法的对象。

In [4]:
class DownCounter

  def initialize(value=10)
    @value = value
  end
  
  def each
    @value.downto(0) do|i|
      yield i
    end
  end
  
end

counter = DownCounter.new(5)

counter.each do|n|
  puts n
end

for n in counter
  puts n
end

5
4
3
2
1
0
5
4
3
2
1
0


5

如果需要对 `DownCounter` 对象遍历的时候，同时获得元素下标，这时的对象是默认不支持 `each_with_index` 方法的。

In [5]:
counter.respond_to?(:each_with_index)

false

In [6]:
origin_methods = counter.methods

[:each, :to_json, :instance_of?, :kind_of?, :is_a?, :tap, :public_send, :method, :public_method, :singleton_method, :remove_instance_variable, :define_singleton_method, :instance_variable_set, :extend, :lsmagic, :to_enum, :enum_for, :gem, :<=>, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :object_id, :send, :display, :to_s, :nil?, :hash, :class, :singleton_class, :clone, :dup, :itself, :taint, :tainted?, :untaint, :untrust, :untrusted?, :trust, :frozen?, :methods, :singleton_methods, :protected_methods, :private_methods, :public_methods, :instance_variable_get, :instance_variables, :instance_variable_defined?, :!, :==, :!=, :__send__, :equal?, :instance_eval, :instance_exec, :__id__]

在 **Ruby** 中，当我们实现一个支持迭代方法的类型的时候，一般都需要继承 `Enumerable`，以便获取更多实用方法。

In [7]:
class DownCounter
  include Enumerable
end

counter.methods - origin_methods

[:max, :min, :to_a, :to_h, :include?, :find, :entries, :sort, :sort_by, :grep, :grep_v, :count, :detect, :find_index, :find_all, :select, :reject, :collect, :map, :flat_map, :collect_concat, :inject, :reduce, :partition, :group_by, :first, :all?, :any?, :one?, :none?, :minmax, :min_by, :max_by, :minmax_by, :member?, :each_with_index, :reverse_each, :each_entry, :each_slice, :each_cons, :each_with_object, :zip, :take, :take_while, :drop, :drop_while, :cycle, :chunk, :slice_before, :slice_after, :slice_when, :chunk_while, :sum, :uniq, :lazy, :to_set]

In [8]:
counter.each_with_index do|n, i|
  puts "pos #{i} is #{n}"
end

pos 0 is 5
pos 1 is 4
pos 2 is 3
pos 3 is 2
pos 4 is 1
pos 5 is 0


#<DownCounter:0x007ffc418d2968 @value=5>

In [9]:
puts counter.max
puts counter.min
puts counter.first
counter.reverse_each{|n| puts "reverse travel #{n}"}
puts counter.collect{|n| 2 * n}

5
0
5
reverse travel 0
reverse travel 1
reverse travel 2
reverse travel 3
reverse travel 4
reverse travel 5
[10, 8, 6, 4, 2, 0]


再来看一个例子

In [10]:
class Collatz
  include Enumerable
    
  def initialize(start)
    @value = start
  end
  
  def each
    v = @value
    loop do
      if v == 1
        break
      elsif v % 2 == 0
        v = v / 2
      else
        v = 3 * v + 1
      end
      yield v
    end
  end
  
end

:each

这里我们实现 [Collatz 猜想](https://zh.wikipedia.org/zh-hans/%E8%80%83%E6%8B%89%E5%85%B9%E7%8C%9C%E6%83%B3)：

- 奇数 n：返回 3n + 1
- 偶数 n：返回 n / 2

直到 n 为 1 为止：

In [11]:
Collatz.new(7).each do|n|
  print "#{n},"
end

22,11,34,17,52,26,13,40,20,10,5,16,8,4,2,1,

## 更多用法

In [12]:
3.times do |i|
  potatoes = i+1
  puts "#{potatoes} potato"
end

1 potato
2 potato
3 potato


3

In [13]:
["apple", "banana", "cherry"].map{|fruit| fruit.reverse}.inject(''){|s, item| s << "<<#{item}>>"}

"<<elppa>><<ananab>><<yrrehc>>"

注意：`each`返回数组本身，链式调用的时候，会忽略代码块的返回值。

In [14]:
["apple", "banana", "cherry"].each{|fruit| fruit.reverse}.inject(''){|s, item| s << "<<#{item}>>"}

"<<apple>><<banana>><<cherry>>"